# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = set(text)
    vocab_to_int = {word: i for i, word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    process = {'.' : '|Punctuation|', ',' : '|Comma|', '?' : '|Question_mark|', '"' : '|Quotation_mark|', ';' :'|Semicolon|', '(' : '|left_paren|', ')' : '|right_paren|', '--' : '|Dash|', '\n' : '|Return|', '!' : '|Exclamation|'}
    
    return process
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input_ = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets_ = tf.placeholder(tf.int32, [None, None], name = 'targets')
    learning_rate_ = tf.placeholder(tf.float32, shape = (), name = 'learning_rate')
    return input_, targets_, learning_rate_


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    rnn_cell = tf.contrib.rnn.MultiRNNCell([lstm]*1)
    initial_state = rnn_cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name = 'initial_state')
    return rnn_cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    input_data = tf.nn.embedding_lookup(embedding, input_data)
    return input_data


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype = tf.float32)
    final_state = tf.identity(state, name = 'final_state')
    return outputs, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed_data = get_embed(input_data, vocab_size, embed_dim)
    outputs, state = build_rnn(cell = cell, inputs = embed_data)
    weights_init = tf.truncated_normal_initializer(stddev=0.1)
    bias_init = tf.zeros_initializer()
    logits = tf.contrib.layers.fully_connected(outputs, num_outputs = vocab_size, activation_fn = None, weights_initializer = weights_init, biases_initializer = bias_init)
    print(logits)
    return logits, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tensor("fully_connected/BiasAdd:0", shape=(128, 5, 27), dtype=float32)
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    char_per_batch = batch_size * seq_length
    num_batch = len(int_text)//char_per_batch
    
    print('batch_size : %s'%batch_size)
    print('seq_length : %s'%seq_length)
    print('Number of batches : %s'%num_batch)
    
    input_text = np.array(int_text[:(num_batch*char_per_batch)])
    target_text = np.array(int_text[1:(num_batch * char_per_batch) + 1])

    target_text[-1] = input_text[0]
    print(input_text)
    print(target_text)
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    #input_ = input_text.reshape(batch_size, -1)
    #target_ = target_text.reshape(batch_size -1)

    #input_ = np.split(input_, num_batch, -1)
    #target_ = np.split(target_, num_batch, -1)
    
    batches = np.array(list(zip(x_batches, y_batches)))

    return batches

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

batch_size : 128
seq_length : 5
Number of batches : 7
[   0    1    2 ..., 4477 4478 4479]
[   1    2    3 ..., 4478 4479    0]
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 69
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.009
# Show stats for every n number of batches
show_every_n_batches = 2

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

Tensor("fully_connected/BiasAdd:0", shape=(?, ?, 6779), dtype=float32)


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

batch_size : 32
seq_length : 16
Number of batches : 134
[4647 1948 2929 ...,  382 1948 1997]
[1948 2929   24 ..., 1948 1997 4647]
Epoch   0 Batch    0/134   train_loss = 8.833
Epoch   0 Batch    2/134   train_loss = 7.223
Epoch   0 Batch    4/134   train_loss = 6.603
Epoch   0 Batch    6/134   train_loss = 6.257
Epoch   0 Batch    8/134   train_loss = 6.029
Epoch   0 Batch   10/134   train_loss = 6.184
Epoch   0 Batch   12/134   train_loss = 5.955
Epoch   0 Batch   14/134   train_loss = 6.072
Epoch   0 Batch   16/134   train_loss = 5.658
Epoch   0 Batch   18/134   train_loss = 5.774
Epoch   0 Batch   20/134   train_loss = 5.611
Epoch   0 Batch   22/134   train_loss = 5.332
Epoch   0 Batch   24/134   train_loss = 5.562
Epoch   0 Batch   26/134   train_loss = 5.437
Epoch   0 Batch   28/134   train_loss = 5.267
Epoch   0 Batch   30/134   train_loss = 5.631
Epoch   0 Batch   32/134   train_loss = 5.275
Epoch   0 Batch   34/134   train_loss = 5.461
Epoch   0 Batch   36/134   train_loss = 5.

Epoch   2 Batch   88/134   train_loss = 3.618
Epoch   2 Batch   90/134   train_loss = 3.643
Epoch   2 Batch   92/134   train_loss = 3.652
Epoch   2 Batch   94/134   train_loss = 3.787
Epoch   2 Batch   96/134   train_loss = 3.678
Epoch   2 Batch   98/134   train_loss = 3.521
Epoch   2 Batch  100/134   train_loss = 3.686
Epoch   2 Batch  102/134   train_loss = 3.533
Epoch   2 Batch  104/134   train_loss = 3.603
Epoch   2 Batch  106/134   train_loss = 3.411
Epoch   2 Batch  108/134   train_loss = 3.485
Epoch   2 Batch  110/134   train_loss = 3.493
Epoch   2 Batch  112/134   train_loss = 3.548
Epoch   2 Batch  114/134   train_loss = 3.483
Epoch   2 Batch  116/134   train_loss = 3.455
Epoch   2 Batch  118/134   train_loss = 3.521
Epoch   2 Batch  120/134   train_loss = 3.586
Epoch   2 Batch  122/134   train_loss = 3.427
Epoch   2 Batch  124/134   train_loss = 3.329
Epoch   2 Batch  126/134   train_loss = 3.436
Epoch   2 Batch  128/134   train_loss = 3.488
Epoch   2 Batch  130/134   train_l

Epoch   5 Batch   46/134   train_loss = 2.515
Epoch   5 Batch   48/134   train_loss = 2.701
Epoch   5 Batch   50/134   train_loss = 2.650
Epoch   5 Batch   52/134   train_loss = 2.717
Epoch   5 Batch   54/134   train_loss = 2.650
Epoch   5 Batch   56/134   train_loss = 2.510
Epoch   5 Batch   58/134   train_loss = 2.613
Epoch   5 Batch   60/134   train_loss = 2.614
Epoch   5 Batch   62/134   train_loss = 2.558
Epoch   5 Batch   64/134   train_loss = 2.580
Epoch   5 Batch   66/134   train_loss = 2.445
Epoch   5 Batch   68/134   train_loss = 2.538
Epoch   5 Batch   70/134   train_loss = 2.619
Epoch   5 Batch   72/134   train_loss = 2.474
Epoch   5 Batch   74/134   train_loss = 2.497
Epoch   5 Batch   76/134   train_loss = 2.435
Epoch   5 Batch   78/134   train_loss = 2.390
Epoch   5 Batch   80/134   train_loss = 2.335
Epoch   5 Batch   82/134   train_loss = 2.433
Epoch   5 Batch   84/134   train_loss = 2.659
Epoch   5 Batch   86/134   train_loss = 2.488
Epoch   5 Batch   88/134   train_l

Epoch   8 Batch    4/134   train_loss = 1.868
Epoch   8 Batch    6/134   train_loss = 1.890
Epoch   8 Batch    8/134   train_loss = 1.925
Epoch   8 Batch   10/134   train_loss = 1.945
Epoch   8 Batch   12/134   train_loss = 1.917
Epoch   8 Batch   14/134   train_loss = 1.879
Epoch   8 Batch   16/134   train_loss = 1.919
Epoch   8 Batch   18/134   train_loss = 1.893
Epoch   8 Batch   20/134   train_loss = 2.004
Epoch   8 Batch   22/134   train_loss = 1.993
Epoch   8 Batch   24/134   train_loss = 2.049
Epoch   8 Batch   26/134   train_loss = 1.911
Epoch   8 Batch   28/134   train_loss = 1.991
Epoch   8 Batch   30/134   train_loss = 1.851
Epoch   8 Batch   32/134   train_loss = 2.029
Epoch   8 Batch   34/134   train_loss = 2.089
Epoch   8 Batch   36/134   train_loss = 1.861
Epoch   8 Batch   38/134   train_loss = 1.885
Epoch   8 Batch   40/134   train_loss = 1.892
Epoch   8 Batch   42/134   train_loss = 1.838
Epoch   8 Batch   44/134   train_loss = 1.833
Epoch   8 Batch   46/134   train_l

Epoch  10 Batch   96/134   train_loss = 1.591
Epoch  10 Batch   98/134   train_loss = 1.610
Epoch  10 Batch  100/134   train_loss = 1.421
Epoch  10 Batch  102/134   train_loss = 1.588
Epoch  10 Batch  104/134   train_loss = 1.562
Epoch  10 Batch  106/134   train_loss = 1.597
Epoch  10 Batch  108/134   train_loss = 1.604
Epoch  10 Batch  110/134   train_loss = 1.463
Epoch  10 Batch  112/134   train_loss = 1.604
Epoch  10 Batch  114/134   train_loss = 1.542
Epoch  10 Batch  116/134   train_loss = 1.548
Epoch  10 Batch  118/134   train_loss = 1.625
Epoch  10 Batch  120/134   train_loss = 1.459
Epoch  10 Batch  122/134   train_loss = 1.427
Epoch  10 Batch  124/134   train_loss = 1.440
Epoch  10 Batch  126/134   train_loss = 1.720
Epoch  10 Batch  128/134   train_loss = 1.493
Epoch  10 Batch  130/134   train_loss = 1.386
Epoch  10 Batch  132/134   train_loss = 1.565
Epoch  11 Batch    0/134   train_loss = 1.416
Epoch  11 Batch    2/134   train_loss = 1.538
Epoch  11 Batch    4/134   train_l

...

Epoch  56 Batch   52/134   train_loss = 0.615
Epoch  56 Batch   54/134   train_loss = 0.607
Epoch  56 Batch   56/134   train_loss = 0.519
Epoch  56 Batch   58/134   train_loss = 0.533
Epoch  56 Batch   60/134   train_loss = 0.658
Epoch  56 Batch   62/134   train_loss = 0.519
Epoch  56 Batch   64/134   train_loss = 0.516
Epoch  56 Batch   66/134   train_loss = 0.614
Epoch  56 Batch   68/134   train_loss = 0.619
Epoch  56 Batch   70/134   train_loss = 0.586
Epoch  56 Batch   72/134   train_loss = 0.596
Epoch  56 Batch   74/134   train_loss = 0.567
Epoch  56 Batch   76/134   train_loss = 0.608
Epoch  56 Batch   78/134   train_loss = 0.582
Epoch  56 Batch   80/134   train_loss = 0.606
Epoch  56 Batch   82/134   train_loss = 0.551
Epoch  56 Batch   84/134   train_loss = 0.669
Epoch  56 Batch   86/134   train_loss = 0.525
Epoch  56 Batch   88/134   train_loss = 0.546
Epoch  56 Batch   90/134   train_loss = 0.588
Epoch  56 Batch   92/134   train_loss = 0.557
Epoch  56 Batch   94/134   train_l

Epoch  59 Batch   10/134   train_loss = 0.502
Epoch  59 Batch   12/134   train_loss = 0.544
Epoch  59 Batch   14/134   train_loss = 0.482
Epoch  59 Batch   16/134   train_loss = 0.475
Epoch  59 Batch   18/134   train_loss = 0.515
Epoch  59 Batch   20/134   train_loss = 0.608
Epoch  59 Batch   22/134   train_loss = 0.586
Epoch  59 Batch   24/134   train_loss = 0.661
Epoch  59 Batch   26/134   train_loss = 0.559
Epoch  59 Batch   28/134   train_loss = 0.411
Epoch  59 Batch   30/134   train_loss = 0.470
Epoch  59 Batch   32/134   train_loss = 0.510
Epoch  59 Batch   34/134   train_loss = 0.464
Epoch  59 Batch   36/134   train_loss = 0.429
Epoch  59 Batch   38/134   train_loss = 0.454
Epoch  59 Batch   40/134   train_loss = 0.505
Epoch  59 Batch   42/134   train_loss = 0.495
Epoch  59 Batch   44/134   train_loss = 0.425
Epoch  59 Batch   46/134   train_loss = 0.488
Epoch  59 Batch   48/134   train_loss = 0.443
Epoch  59 Batch   50/134   train_loss = 0.530
Epoch  59 Batch   52/134   train_l

Epoch  61 Batch  102/134   train_loss = 0.497
Epoch  61 Batch  104/134   train_loss = 0.450
Epoch  61 Batch  106/134   train_loss = 0.431
Epoch  61 Batch  108/134   train_loss = 0.457
Epoch  61 Batch  110/134   train_loss = 0.431
Epoch  61 Batch  112/134   train_loss = 0.563
Epoch  61 Batch  114/134   train_loss = 0.472
Epoch  61 Batch  116/134   train_loss = 0.506
Epoch  61 Batch  118/134   train_loss = 0.523
Epoch  61 Batch  120/134   train_loss = 0.405
Epoch  61 Batch  122/134   train_loss = 0.392
Epoch  61 Batch  124/134   train_loss = 0.455
Epoch  61 Batch  126/134   train_loss = 0.496
Epoch  61 Batch  128/134   train_loss = 0.525
Epoch  61 Batch  130/134   train_loss = 0.441
Epoch  61 Batch  132/134   train_loss = 0.441
Epoch  62 Batch    0/134   train_loss = 0.475
Epoch  62 Batch    2/134   train_loss = 0.490
Epoch  62 Batch    4/134   train_loss = 0.465
Epoch  62 Batch    6/134   train_loss = 0.487
Epoch  62 Batch    8/134   train_loss = 0.506
Epoch  62 Batch   10/134   train_l

Epoch  64 Batch   60/134   train_loss = 0.581
Epoch  64 Batch   62/134   train_loss = 0.571
Epoch  64 Batch   64/134   train_loss = 0.444
Epoch  64 Batch   66/134   train_loss = 0.594
Epoch  64 Batch   68/134   train_loss = 0.541
Epoch  64 Batch   70/134   train_loss = 0.650
Epoch  64 Batch   72/134   train_loss = 0.506
Epoch  64 Batch   74/134   train_loss = 0.615
Epoch  64 Batch   76/134   train_loss = 0.608
Epoch  64 Batch   78/134   train_loss = 0.501
Epoch  64 Batch   80/134   train_loss = 0.545
Epoch  64 Batch   82/134   train_loss = 0.559
Epoch  64 Batch   84/134   train_loss = 0.698
Epoch  64 Batch   86/134   train_loss = 0.483
Epoch  64 Batch   88/134   train_loss = 0.527
Epoch  64 Batch   90/134   train_loss = 0.591
Epoch  64 Batch   92/134   train_loss = 0.558
Epoch  64 Batch   94/134   train_loss = 0.588
Epoch  64 Batch   96/134   train_loss = 0.619
Epoch  64 Batch   98/134   train_loss = 0.598
Epoch  64 Batch  100/134   train_loss = 0.557
Epoch  64 Batch  102/134   train_l

Epoch  67 Batch   18/134   train_loss = 0.997
Epoch  67 Batch   20/134   train_loss = 1.239
Epoch  67 Batch   22/134   train_loss = 1.210
Epoch  67 Batch   24/134   train_loss = 1.220
Epoch  67 Batch   26/134   train_loss = 1.035
Epoch  67 Batch   28/134   train_loss = 0.996
Epoch  67 Batch   30/134   train_loss = 1.016
Epoch  67 Batch   32/134   train_loss = 1.103
Epoch  67 Batch   34/134   train_loss = 0.986
Epoch  67 Batch   36/134   train_loss = 0.932
Epoch  67 Batch   38/134   train_loss = 0.951
Epoch  67 Batch   40/134   train_loss = 1.043
Epoch  67 Batch   42/134   train_loss = 1.056
Epoch  67 Batch   44/134   train_loss = 0.890
Epoch  67 Batch   46/134   train_loss = 1.096
Epoch  67 Batch   48/134   train_loss = 1.098
Epoch  67 Batch   50/134   train_loss = 1.054
Epoch  67 Batch   52/134   train_loss = 1.037
Epoch  67 Batch   54/134   train_loss = 1.149
Epoch  67 Batch   56/134   train_loss = 1.096
Epoch  67 Batch   58/134   train_loss = 1.073
Epoch  67 Batch   60/134   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    pick = np.random.choice(list(int_to_vocab.values()),p=probabilities)
    return pick 

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [21]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: hey, i don't know, ever since obama came in the eyes!
homer_simpson:(taunting) hey, barney? my beautiful, beautiful midge) marge, i need a little more" back of a drink?
moe_szyslak: no.
glen: do we put in the prayers.
chief_wiggum: your daughter no man enough to survive on bed i'll show you talk me outta seein' edna, i will say good, i'd look like to be kept behind a secret.
homer_simpson:(loud) did you say to me.
homer_simpson: i'd like to introduce ned flanders, when it gets to a cow college him?
moe_szyslak: ummmmmmmmm? uh-oh. i don't think so hard? somebody!
homer_simpson: hey moe! don't!
moe_szyslak:(hurt sigh) i'm on my dad.
moe_szyslak: if it isn't little miss" how i loathe him.
carl_carlson:(singing) son, say hello to commit a crime, moe. you still got us.
moe_szyslak: just do is either with the way we did in the game.


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.